In [1]:
import neuron
usr_nm = os.path.expanduser('~')
neuron.load_mechanisms(usr_nm + '/Documents/NEURON_mechanism/')
from neuron import h, gui
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from scipy.signal import find_peaks
from sklearn.svm import SVC
import random 
import time
from sklearn.linear_model import LinearRegression

In [2]:
font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 6}

plt.rc('font', **font)
plt.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
run general_fun.ipynb

In [4]:
h.xopen("MG_reconstructed_by_Nate_Sawtell");

len_seg = 30
for sec in h.all:
    sec.nseg = int(np.ceil(sec.L/len_seg))
    if not (sec.nseg % 2):
        sec.nseg = sec.nseg + 1
        
vml_thresh = 100
h.distance(sec=h.soma)

vml = h.SectionList()
dml = h.SectionList()
exclude_comp = ['apic[0]']
for sec in h.apical:
    if sec.hname() not in exclude_comp:
        dist_from_soma = h.distance(sec(.5))
        if dist_from_soma < vml_thresh:
            vml.append(sec=sec)
        elif dist_from_soma >= vml_thresh:
            dml.append(sec=sec)

apic_soma = h.SectionList()  
for sec in h.apical:
    apic_soma.append(sec=sec)
for sec in h.somatic:
    apic_soma.append(sec=sec)
        
h.celsius = 20
set_init = -65
h.v_init = set_init

spe_cond = 0.0003
capac = 1
# insert channels

for sec in h.all:
    sec.Ra = 100    # Axial resistance in Ohm * cm
    sec.cm = capac      # Membrane capacitance in micro Farads / cm^2
for sec in h.basal:
        sec.insert('pas')
        sec.g_pas = spe_cond
        sec.e_pas = set_init

In [5]:
def make_bse(TTX = False, TTX_axon = False, TTX_apical = False, set_init = -65):

    axon_no_hh = ['axon[0]']
    for sec in h.axonal:
        if any(word in sec.hname() for word in axon_no_hh):  
#             sec.insert('pas')
#             sec.g_pas = spe_cond
#             sec.e_pas = set_init
            sec.insert('hh')
            sec.gl_hh = spe_cond
            sec.el_hh = set_init
            if TTX or TTX_axon:      
                sec.gnabar_hh = 0
                sec.gkbar_hh =  0
            else:
                sec.gnabar_hh = 0.168 # 2
                sec.gkbar_hh =  0.05  # 0.2
        else:
            sec.insert('hh')
            sec.gl_hh = spe_cond
            sec.el_hh = set_init
            if TTX or TTX_axon:      
                sec.gnabar_hh = 0
                sec.gkbar_hh =  0
            else:
                sec.gnabar_hh = 4 # 2
                sec.gkbar_hh =  0.5  # 0.2
#                 sec.gbar_Kv1_1 = 30
    #         sec.insert('Kdr')
    #         sec.insert('NaF')
    #         for seg in sec:
    #             if TTX or TTX_axon:
    #                 seg.NaF.gnabar = 0
    #             else:
    #                 seg.NaF.gnabar = 0 # 0.1
    #             seg.Kdr.gkbar = 0   # 0.5 Potassium conductance in S/cm2


    apic_no_syn = ['apic[0]'] #,'apic[12]','apic[13]'
#     apic_no_TTX = ['apic[1]']
    for sec in h.apical:
        if sec.hname() in apic_no_syn:
            sec.insert('pas')
            sec.g_pas = spe_cond
            sec.e_pas = set_init
        else:  
            sec.insert('hh')
            sec.gl_hh = spe_cond
            sec.el_hh = set_init  
#             sec.insert('cal') 
            if TTX or TTX_apical: # and sec.hname() not in apic_no_TTX      
                sec.gnabar_hh = 0
                sec.gkbar_hh =  0
#                 sec.gcalbar_cal =  0 
            else:
                sec.gnabar_hh = 0.1 #   7C 0.033
                sec.gkbar_hh =  0.008 #   7C 0.007
#                 sec.gcalbar_cal =  0.075


In [6]:
make_bse(TTX = False, TTX_axon = False, TTX_apical = False, set_init = -65)

In [7]:
def run_sim(trial_length = 100, Inh = False, pf_exc_canc_inh = False, Nspk_up_Bspk_down = False,\
            Inh_gmax = 0.04, somatic_inh = False, apical_inh = False, canc_gmax = .00022, stim_dur = 5000, record_conduc = False,\
           continuous_sensory = False, can_opposite = False):


    stim = []
    stim_all = []
    skip_every = 20*0.025
    dt = 0.025
    ampl_noise = 1/(np.sqrt(dt/4))*np.random.normal(0, .0057, int(stim_dur/skip_every)+1)
#     ampl_noise = np.random.lognormal(-3.35, .95, stim_dur)
    ampl_noise[0:200] = 0
    idx = 1;
    for i in np.arange(0,stim_dur,skip_every):
        stim = h.IClamp(h.soma(0.5))
        stim.delay = i
        stim.dur = skip_every
        stim.amp = ampl_noise[idx]
        idx = idx+1
        stim_all.append(stim)

        
    if continuous_sensory:
        if Inh:    
            gaba = []
            gaba_all = [] 
            exclude_comp = ['dend[0]']
            for sec in h.dend:
                if sec.hname() not in exclude_comp:
                    for ii in np.arange(10,stim_dur,10):
                        gaba = h.GabaASyn(sec(0.45)) 
                        gaba.onset = ii + np.random.normal(0, 5, 1) 
                        gaba.gmax = Inh_gmax + .001*np.random.normal(1, 1, 1) # 0.09 
                        gaba_all.append(gaba)
                
        if pf_exc_canc_inh: 
            ampaAB = []
            ampaAB_all = []
            exclude_comp = ['apic[0]']
            for sec in h.apical:
                if sec.hname() not in exclude_comp:
   
                    for ii in np.arange(10,stim_dur,10):
                        ampa_AB = h.AmpaSyn(sec(0.5))  
                        ampa_AB.onset = ii + np.random.normal(0, 5, 1) # + np.random.normal(1, 3, 1) (0.2, 3, 1)
                        ampa_AB.gmax = canc_gmax + .00002*np.random.normal(1, 2, 1) # .00021 + .00002*np.random.normal(1, 2, 1)
                        ampaAB_all.append(ampa_AB)

        if Nspk_up_Bspk_down:
            gaba = []
            gaba_all = []    
            exclude_comp = ['dend[0]']
            for sec in h.dend:
                if sec.hname() not in exclude_comp:
                    for ii in np.arange(10,stim_dur,10):
                        gaba = h.GabaASyn(sec(0.45)) 
                        gaba.onset = ii + np.random.normal(0, 5, 1) 
                        gaba.gmax = 0.18 + .001*np.random.normal(1, 1, 1) # 0.09 
                        gaba_all.append(gaba)
                    
            ampaABE = []
            ampaABE_all = []
            for sec in h.dend:
                for ii in np.arange(10,stim_dur,10):
                    ampa_ABE = h.AmpaSyn(sec(0.5)) 
                    ampa_ABE.onset = ii + np.random.normal(0, 5, 1)
                    ampa_ABE.gmax = 0.001 + .00001*np.random.normal(1, 1, 1) #+ .01*np.random.normal(0, 1, 1)   #0.013
                    ampaABE_all.append(ampa_ABE)
                    
            if can_opposite:
                ampaAB = []
                ampaAB_all = []
                exclude_comp = ['apic[0]']
                for sec in h.apical:
                    if sec.hname() not in exclude_comp:
                        for ii in np.arange(10,stim_dur,10):
                            ampa_AB = h.AmpaSyn(sec(0.5))  
                            ampa_AB.onset = ii + np.random.normal(0, 5, 1) # + np.random.normal(1, 3, 1) (0.2, 3, 1)
                            ampa_AB.gmax = canc_gmax 
                            ampaAB_all.append(ampa_AB)

                    
        if somatic_inh:    
            gaba = []
            gaba_all = []
            for ii in np.arange(10,stim_dur,10):
                gaba = h.GabaASyn(h.soma(0.4))
                gaba.onset = ii + np.random.normal(0, 5, 1) 
                gaba.gmax = Inh_gmax + .001*np.random.normal(1, 1, 1) # 0.09 
                gaba_all.append(gaba)
        
        if apical_inh:   
            gaba = []
            gaba_all = [] 
            exclude_comp = ['apic[0]']
            for sec in vml:
                if sec.hname() not in exclude_comp:
                    for ii in np.arange(10,stim_dur,10):
                        gaba = h.GabaASyn(sec(0.45)) 
                        gaba.onset = ii + np.random.normal(0, 5, 1) 
                        gaba.gmax = Inh_gmax + .001*np.random.normal(1, 1, 1) # 0.09 
                        gaba_all.append(gaba)
#######             
    else:
        if Inh:    
            gaba = []
            gaba_all = [] 
            for sec in vml:
                for ii in np.arange((trial_length/2-3),stim_dur,trial_length):
                    gaba = h.GabaASyn(sec(0.45)) 
                    gaba.onset = ii + np.random.normal(0, 5, 1) 
                    gaba.gmax = Inh_gmax + .001*np.random.normal(1, 1, 1) # 0.09 
                    gaba_all.append(gaba)

        if pf_exc_canc_inh: 
            ampaAB = []
            ampaAB_all = []
            exclude_comp = ['apic[0]']
            for sec in h.apical:
                if sec.hname() not in exclude_comp:   
                    for ii in np.arange((trial_length/2-3),stim_dur,trial_length):
                        ampa_AB = h.AmpaSyn(sec(0.5))  
                        ampa_AB.onset = ii + np.random.normal(0, 5, 1) # + np.random.normal(1, 3, 1) (0.2, 3, 1)
                        ampa_AB.gmax = canc_gmax + .00002*np.random.normal(1, 2, 1) # .00021 + .00002*np.random.normal(1, 2, 1)
                        ampaAB_all.append(ampa_ABE)
######
        if Nspk_up_Bspk_down:
            gaba = []
            gaba_all = []    
            for sec in vml:
                for ii in np.arange((trial_length/2-3),stim_dur,trial_length):
                    gaba = h.GabaASyn(sec(0.45)) 
                    gaba.onset = ii + np.random.normal(0, 2, 1) 
                    gaba.gmax = 0.2 + .001*np.random.normal(1, 1, 1) # strong 0.05 -- small 0.025 -- Miniscule 0.01
                    gaba_all.append(gaba)

            ampaABE = []
            ampaABE_all = []  
            for ii in np.arange((trial_length/2),stim_dur,trial_length):
                ampa_ABE = h.AmpaSyn(h.dend[1](0.5)) 
                ampa_ABE.onset = ii + np.random.normal(0, 4, 1) 
                ampa_ABE.gmax = 0.02 + .001*np.random.normal(1, 1, 1) #+ .01*np.random.normal(0, 1, 1)   #0.013
                ampaABE_all.append(ampa_AB)


    dict_all = {}
    v_vec_apic = h.Vector()        
    v_vec_apic_up = h.Vector()
    v_vec_axon = h.Vector()        
    v_vec_soma = h.Vector()        
    t_vec = h.Vector()             

    v_vec_apic.record(h.apic[11](0.5)._ref_v)
    v_vec_apic_up.record(h.apic[11](1)._ref_v)
    v_vec_axon.record(h.axon[3](0.5)._ref_v)
    v_vec_soma.record(h.soma(0.5)._ref_v)
    t_vec.record(h._ref_t)
    
    if record_conduc:
        ina_vec_apic = h.Vector()        
        ik_vec_apic = h.Vector() 
        m_vec_apic = h.Vector()        
        h_vec_apic = h.Vector() 
        n_vec_apic = h.Vector() 

        ina_vec_apic.record(h.apic[11](1)._ref_ina)
        ik_vec_apic.record(h.apic[11](1)._ref_ik)
        m_vec_apic.record(h.apic[11](1)._ref_m_hh)
        h_vec_apic.record(h.apic[11](1)._ref_h_hh)
        n_vec_apic.record(h.apic[11](1)._ref_n_hh)

    h.tstop = stim_dur
    h.run()
    dict_all['t_vec'] =  t_vec
    dict_all['v_vec_apic'] = v_vec_apic
    dict_all['v_vec_apic_up'] = v_vec_apic_up
    dict_all['v_vec_axon'] = v_vec_axon
    dict_all['v_vec_soma'] = v_vec_soma
    
    if record_conduc:
        dict_all['ina_vec_apic'] = ina_vec_apic        
        dict_all['ik_vec_apic'] = ik_vec_apic 
        dict_all['m_vec_apic'] = m_vec_apic        
        dict_all['h_vec_apic'] = h_vec_apic 
        dict_all['n_vec_apic'] = n_vec_apic 

    stim = []
    stim_all = []

    ampaA = []
    ampaA_all = []

    ampaAAB = []
    ampaAAB_all = []

    gaba = []
    gaba_all = []

    ampaAB = []
    ampaAB_all = []

    ampaABE = []
    ampaABE_all = []

    return dict_all

In [8]:
t = time.time()
np.random.seed(seed=1)
nr_runs = 100
stim_dur = 5000
trial_length = 100
eliminate_trials = 3
record_conduc = True
continuous_sensory = True

Inh = False
Inh_gmax = 0 #0.03 
pf_exc_canc_inh = False
canc_gmax = 0 #.000083
Nspk_up_Bspk_down = False
after_minus_before = False

if Nspk_up_Bspk_down:
    Inh = False
    pf_exc_canc_inh = False
    
if pf_exc_canc_inh:
    Inh = True
    Nspk_up_Bspk_down = False

if after_minus_before:
    Inh = False
    pf_exc_canc_inh = True
    Nspk_up_Bspk_down = False
    
all_runs = []
all_runs_apic = []
all_runs_apic_up = []
all_runs_axon = []

all_runs_ina = []
all_runs_ik = []
all_runs_m = []
all_runs_h = []
all_runs_n = []

for ii in range(nr_runs):
    results_all = run_sim(trial_length = trial_length, Inh = Inh, Inh_gmax = Inh_gmax, pf_exc_canc_inh = pf_exc_canc_inh,\
                          canc_gmax = canc_gmax, Nspk_up_Bspk_down = Nspk_up_Bspk_down,\
                          stim_dur = stim_dur, continuous_sensory = continuous_sensory, record_conduc = record_conduc)
    all_trials,trial_length_dt = trace_to_trials(results_all['v_vec_soma'].to_python(),eliminate_trials = eliminate_trials,\
                                                 trial_length = trial_length)
    all_trials_apic,trial_length_dt = trace_to_trials(results_all['v_vec_apic'].to_python(),eliminate_trials = eliminate_trials,\
                                                 trial_length = trial_length)
    all_trials_apic_up,trial_length_dt = trace_to_trials(results_all['v_vec_apic_up'].to_python(),eliminate_trials = eliminate_trials,\
                                                 trial_length = trial_length)
    all_trials_axon,trial_length_dt = trace_to_trials(results_all['v_vec_axon'].to_python(),eliminate_trials = eliminate_trials,\
                                                 trial_length = trial_length)

    all_runs = all_runs+all_trials
    all_runs_apic = all_runs_apic+all_trials_apic
    all_runs_apic_up = all_runs_apic_up+all_trials_apic_up
    all_runs_axon = all_runs_axon+all_trials_axon

    if record_conduc:
        all_trials_ina,trial_length_dt = trace_to_trials(results_all['ina_vec_apic'].to_python(),eliminate_trials = eliminate_trials,\
                                                 trial_length = trial_length)
        all_trials_ik,trial_length_dt = trace_to_trials(results_all['ik_vec_apic'].to_python(),eliminate_trials = eliminate_trials,\
                                                     trial_length = trial_length)
        all_trials_m,trial_length_dt = trace_to_trials(results_all['m_vec_apic'].to_python(),eliminate_trials = eliminate_trials,\
                                                     trial_length = trial_length)
        all_trials_h,trial_length_dt = trace_to_trials(results_all['h_vec_apic'].to_python(),eliminate_trials = eliminate_trials,\
                                                     trial_length = trial_length)
        all_trials_n,trial_length_dt = trace_to_trials(results_all['n_vec_apic'].to_python(),eliminate_trials = eliminate_trials,\
                                                     trial_length = trial_length)
    
    
        all_runs_ina = all_runs_ina+all_trials_ina
        all_runs_ik = all_runs_ik+all_trials_ik
        all_runs_m = all_runs_m+all_trials_m
        all_runs_h = all_runs_h+all_trials_h
        all_runs_n= all_runs_n+all_trials_n
    
all_trials = all_runs

elapsed = time.time() - t
print(elapsed)

379.590696811676


In [9]:
data_baseline_noise = all_runs_apic
data_baseline_noise_apic_up = all_runs_apic_up
data_baseline_noise_soma = all_runs
data_baseline_noise_axon = all_runs_axon
data_baseline_ina = all_runs_ina 
data_baseline_ik = all_runs_ik 
data_baseline_m = all_runs_m 
data_baseline_h = all_runs_h 
data_baseline_n = all_runs_n

spk_times_all = turn_trace_to_spk([np.hstack(all_runs)], min_hight_nspk = -60, min_hight_bspk = -47.5,\
                                  prom_nspk = 3, prom_bspk = 15)

peaks_nspk_loc_baseline = spk_times_all['peaks_nspk_all']
peaks_bspk_loc_baseline = spk_times_all['peaks_bspk_all']
nspk_peak_baseline = [x['peak_heights'] for x in spk_times_all['properties_nspk_all']]

# FI CURVE

In [10]:
data_soma = [np.hstack(data_baseline_noise_soma)]

# data_soma = [np.hstack(data_baseline_noise_soma+data_cancellation_noise_soma+data_inhibition_noise_soma+\
#                       data_soma_inh_noise_soma+data_soma_can_noise_soma)]

spk_times_all = turn_trace_to_spk(data_soma, min_hight_nspk = -60, min_hight_bspk = -47.5,\
                                  prom_nspk = 3, prom_bspk = 15)

ns_pk_non_evoking = []
ns_loc_non_evoking = []
nspk_peaks_pre = []
ns_loc_pre_bs = []
dt = 0.025
nspk_before_bspk = np.round(8/dt)
peaks_nspk_loc = spk_times_all['peaks_nspk_all']
peaks_bspk_loc = spk_times_all['peaks_bspk_all']
nspk_peak = [x['peak_heights'] for x in spk_times_all['properties_nspk_all']]
nspk_peaks_all = np.concatenate(nspk_peak)
if any(peaks_bspk_loc[0]):
    nspk_before_idx_all = []
    for jj in range(len(peaks_bspk_loc[0])):
        nspk_before_idx = np.concatenate(np.where((peaks_nspk_loc[0] > (peaks_bspk_loc[0][jj] - nspk_before_bspk)) & (peaks_nspk_loc[0] < peaks_bspk_loc[0][jj])))
        if len(nspk_before_idx) > 0:
            nspk_before_idx = nspk_before_idx[-1]
            nspk_before_idx_all.append(nspk_before_idx)
idx_all = range(len(nspk_peak[0]))
nspk_non_before_idx = np.delete(idx_all,nspk_before_idx_all) 
nspk_peaks_pre.append(nspk_peak[0][nspk_before_idx_all])
ns_loc_pre_bs.append(peaks_nspk_loc[0][nspk_before_idx_all])
ns_pk_non_evoking.append(nspk_peak[0][nspk_non_before_idx])    
ns_loc_non_evoking.append(peaks_nspk_loc[0][nspk_non_before_idx])  


In [11]:

bin_size = 0.1
nbins = np.ceil((np.max(nspk_peaks_all)-np.min(nspk_peaks_all))/bin_size)
edges = np.linspace(np.min(nspk_peaks_all),np.max(nspk_peaks_all),num = int(nbins+1))
counts_all, bins_all = np.histogram(np.array(nspk_peaks_all,dtype='float'),bins=edges)
counts_pre, bins_pre = np.histogram(np.array(nspk_peaks_pre,dtype='float'),bins=edges)
r_bspk_pk = counts_pre/counts_all
# r_bspk_pk[np.isnan(r_bspk_pk )] = 0

/var/folders/lj/190zt78d6zgbjnk6n21cfk180000gn/T/ipykernel_82419/1753302847.py:6: RuntimeWarning: invalid value encountered in true_divide
  r_bspk_pk = counts_pre/counts_all


In [13]:
%matplotlib osx
fig = plt.figure(figsize=(3*1.1,3*2))
fs = 6
ts = 6
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs)

plt.plot(edges[1:],r_bspk_pk,'ok',markersize = 2)
plt.xlabel('peak potential (mV)',fontsize = fs)
plt.ylabel('p[broad spike]',fontsize = fs)

ax = plt.gca()
ax.tick_params(direction='in', length=2, width=0.5, colors='k',
               grid_color='k', grid_alpha=1)


# AMP analysis

In [15]:
dt = 0.025
time_before_pk = int(np.round(0.7/dt))
min_der = 0.08
add_ind = 1

In [27]:
# %matplotlib osx

fs = 6
ts = 6
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs)

nr_hist = 200
x_range = [-65,-52]
y_range = [0,0.15]


titles = ['baseline','inhib.','cancel.','opposite','oppos_can']
sites = ['soma','apic','apic up']
colr = ['k','r','m']

dt = 0.025
loc_up_apic = 0.12/dt
loc_up_apic_up = 0.25/dt
base_nspk = int(np.round(0.675/dt))


dt = 0.025
nspk_before_bspk = np.round(8/dt)
bin_size = 0.1
min_ind = 1

slope_intercept_all = np.zeros((3,2))
mean_AMP_MP_all = np.zeros((3,2))
mean_AMP_evoke_non_all = np.zeros((3,2))

use_basil_inh = True
use_soma_inh = False
use_apic_inh = False

show_apic = False
fig = plt.figure(figsize=(2,2))

MP_all = np.empty(0)
AMP_all = np.empty(0)
clr_all = np.empty(0)

for kk in range(1):
    
    if kk == 0:
        data_apic_up = [np.hstack(data_baseline_noise_apic_up)]
        data_soma = [np.hstack(data_baseline_noise_soma)]
        peaks_nspk_loc = peaks_nspk_loc_baseline
        peaks_bspk_loc = peaks_bspk_loc_baseline
        nspk_peak = nspk_peak_baseline
    if kk>0:
        if use_basil_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_inhibition_noise_apic_up)]
                data_soma = [np.hstack(data_inhibition_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_inhibition
                peaks_bspk_loc = peaks_bspk_loc_inhibition
                nspk_peak = nspk_peak_inhibition
            elif kk == 2: 
                data_apic_up = [np.hstack(data_cancellation_noise_apic_up)]
                data_soma = [np.hstack(data_cancellation_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_cancellation
                peaks_bspk_loc = peaks_bspk_loc_cancellation
                nspk_peak = nspk_peak_cancellation
        elif use_apic_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_apic_inh_noise_apic_up)]
                data_soma = [np.hstack(data_apic_inh_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_apic_inh
                peaks_bspk_loc = peaks_bspk_loc_apic_inh
                nspk_peak = nspk_peak_apic_inh
            elif kk == 2: 
                data_apic_up = [np.hstack(data_apic_can_noise_apic_up)]
                data_soma = [np.hstack(data_apic_can_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_apic_can
                peaks_bspk_loc = peaks_bspk_loc_apic_can
                nspk_peak = nspk_peak_apic_can
        elif use_soma_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_soma_inh_noise_apic_up)]
                data_soma = [np.hstack(data_soma_inh_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_soma_inh
                peaks_bspk_loc = peaks_bspk_loc_soma_inh
                nspk_peak = nspk_peak_soma_inh
            elif kk == 2: 
                data_apic_up = [np.hstack(data_soma_can_noise_apic_up)]
                data_soma = [np.hstack(data_soma_can_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_soma_can
                peaks_bspk_loc = peaks_bspk_loc_soma_can
                nspk_peak = nspk_peak_soma_can

        
        
    if show_apic:
        if kk == 0 or kk == 1:
            min_pk_nsp = -60
        elif kk == 2:
            min_pk_nsp = -58.5
            
        spk_times_all = turn_trace_to_spk(data_apic_up, min_hight_nspk = min_pk_nsp, min_hight_bspk = -47.5,\
                                          prom_nspk = 1.5, prom_bspk = 15)
        peaks_nspk_loc_apic_up = np.concatenate(spk_times_all['peaks_nspk_all'])
        peaks_bspk_loc = (spk_times_all['peaks_bspk_all'])
        nspk_peaks_all_apic_up = np.concatenate([x['peak_heights'] for x in spk_times_all['properties_nspk_all']])
        bspk_peaks_all = ([x['peak_heights'] for x in spk_times_all['properties_bspk_all']])
        
        data_rel = np.concatenate(data_apic_up)
        PKS = nspk_peaks_all_apic_up
        peaks_loc_rel = peaks_nspk_loc_apic_up    
    
    else:
        data_rel = np.concatenate(data_soma)
                
        PKS = np.concatenate(nspk_peak) 
        peaks_loc_rel = np.concatenate(peaks_nspk_loc)
        
       

    fst_dev = np.diff(data_rel,1)
    fst_dev = np.insert(fst_dev,0, 0)

    mp_nspk_locs = []
    idx_list = []
    for ii in range(len(peaks_loc_rel)):  
        rel_idx = [peaks_loc_rel[ii]-time_before_pk+add_ind+np.where(fst_dev[peaks_loc_rel[ii]-time_before_pk:peaks_loc_rel[ii]]>min_der)[0]]
        if len(rel_idx[0])>0:
            mp_nspk_locs.append(rel_idx[0][0])
        else:
            print(ii)
            idx_list.append(ii)
    PKS = np.delete(PKS,idx_list)  
    peaks_loc_rel = np.delete(peaks_loc_rel,idx_list)  
    
    MP = data_rel[mp_nspk_locs]
    AMP = PKS-MP
    
    mean_AMP_MP_all[kk,0] = np.mean(PKS)
    mean_AMP_MP_all[kk,1] = np.mean(MP)

    ns_pk_non_evoking = []
    ns_mp_non_evoking = []
    ns_amp_non_evoking = []
    ns_loc_non_evoking = []
    ns_pk_pre = []
    ns_mp_pre = []
    ns_amp_pre = []
    ns_loc_pre_bs = []
    if any(peaks_bspk_loc[0]):
        nspk_before_idx_all = []
        for jj in range(len(peaks_bspk_loc[0])):
            nspk_before_idx = np.concatenate(np.where((peaks_loc_rel > (peaks_bspk_loc[0][jj] - nspk_before_bspk)) & (peaks_loc_rel < peaks_bspk_loc[0][jj])))
            if len(nspk_before_idx) > 0:
                nspk_before_idx = nspk_before_idx[-1]
                nspk_before_idx_all.append(nspk_before_idx)
        idx_all = range(len(PKS))
        nspk_non_before_idx = np.delete(idx_all,nspk_before_idx_all) 
        ns_pk_pre.append(PKS[nspk_before_idx_all])
        ns_loc_pre_bs.append(peaks_loc_rel[nspk_before_idx_all])
        ns_pk_non_evoking.append(PKS[nspk_non_before_idx])    
        ns_loc_non_evoking.append(peaks_loc_rel[nspk_non_before_idx])  

        ns_mp_pre.append(MP[nspk_before_idx_all])
        ns_mp_non_evoking.append(MP[nspk_non_before_idx])    
        ns_amp_pre.append(AMP[nspk_before_idx_all])
        ns_amp_non_evoking.append(AMP[nspk_non_before_idx])
        

        ns_mp_pre_conc = np.concatenate(ns_mp_pre);
        ns_amp_pre_conc = np.concatenate(ns_amp_pre);
        ns_mp_non_evoking_conc = np.concatenate(ns_mp_non_evoking)
        ns_amp_non_evoking_conc = np.concatenate(ns_amp_non_evoking)

        nbins = np.ceil((np.max(MP)-np.min(MP))/bin_size)
        edges = np.linspace(np.min(MP),np.max(MP),num = int(nbins+1))
        inds_non_evoke = np.digitize(np.array(ns_mp_non_evoking_conc ,dtype='float'), bins=edges)
        mean_amp_non_evoke = np.empty((np.size(edges)))
        mean_amp_non_evoke[:] = np.NaN
        for ii in range(len(edges)):
            if np.sum(inds_non_evoke == ii) > min_ind:
                mean_amp_non_evoke[ii] =  np.mean(ns_amp_non_evoking_conc[inds_non_evoke == ii])                  
        inds_evoke = np.digitize(np.array(ns_mp_pre_conc ,dtype='float'), bins=edges)
        mean_amp_evoke = np.empty((np.size(edges)))
        mean_amp_evoke[:] = np.NaN
        for ii in range(len(edges)):
            if np.sum(inds_evoke == ii) > min_ind:
                mean_amp_evoke[ii] =  np.mean(ns_amp_pre_conc[inds_evoke == ii])                  
        
        mean_AMP_evoke_non_all[kk,0] = np.mean(ns_amp_pre)
        mean_AMP_evoke_non_all[kk,1] = np.mean(ns_amp_non_evoking)
    else:
        nbins = np.ceil((np.max(MP)-np.min(MP))/bin_size)
        edges = np.linspace(np.min(MP),np.max(MP),num = int(nbins+1))
        counts_all, bins_all = np.histogram(np.array(MP,dtype='float'),bins=edges)
        inds_non_evoke = np.digitize(np.array(MP,dtype='float'), bins=edges)
        mean_amp_non_evoke = np.empty((np.size(edges)))
        mean_amp_non_evoke[:] = np.NaN
        for ii in range(len(edges)):
            if np.sum(inds_non_evoke == ii) > min_ind:
                mean_amp_non_evoke[ii] =  np.mean(AMP[inds_non_evoke == ii])                  
        mean_amp_evoke = np.empty((np.size(edges)))
        mean_amp_evoke[:] = np.NaN
        ind_evoke = np.empty((np.size(edges)))
    
    
    
    X_MP  = np.transpose([MP])
    model_cell = LinearRegression(fit_intercept=True).fit(X_MP, np.transpose(AMP))
    
    plt.scatter(edges[4:54],mean_amp_non_evoke[4:54],s=10, color='k')
    plt.scatter(edges[4:54],mean_amp_evoke[4:54],s=10, facecolors='none', edgecolors='k')
    plt.legend(['non-evoking','evoking'],frameon = False, prop={'size': fs})
    plt.tight_layout()
    plt.ylabel('amplitude (mV)', fontsize = fs)
    plt.xlabel('baseline membrane potential (mV)', fontsize = fs)

    ax = plt.gca()
    ax.tick_params(direction='in', length=2, width=0.5, colors='k',
                   grid_color='k', grid_alpha=1)

plt.show()



In [17]:
print(np.mean(ns_amp_pre))
print(np.mean(ns_amp_non_evoking))


13.483453810895142
12.36405390443785


# PEAK vs MP

In [25]:
%matplotlib osx

fs = 6
ts = 6
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs)

nr_hist = 200
x_range = [-65,-52]
y_range = [0,0.15]


titles = ['baseline','inhib.','cancel.','opposite','oppos_can']
sites = ['soma','apic','apic up']
colr = ['k','r','m']

dt = 0.025
loc_up_apic = 0.12/dt
loc_up_apic_up = 0.25/dt
base_nspk = int(np.round(0.675/dt))


dt = 0.025
nspk_before_bspk = np.round(8/dt)
bin_size = 0.1
min_ind = 2

slope_intercept_all = np.zeros((3,2))
mean_AMP_MP_all = np.zeros((3,2))
mean_AMP_evoke_non_all = np.zeros((3,2))

use_basil_inh = True
use_soma_inh = False
use_apic_inh = False

show_apic = False
fig = plt.figure(figsize=(2,2))

MP_all = np.empty(0)
AMP_all = np.empty(0)
clr_all = np.empty(0)

for kk in range(1):
    
    if kk == 0:
        data_apic_up = [np.hstack(data_baseline_noise_apic_up)]
        data_soma = [np.hstack(data_baseline_noise_soma)]
        peaks_nspk_loc = peaks_nspk_loc_baseline
        peaks_bspk_loc = peaks_bspk_loc_baseline
        nspk_peak = nspk_peak_baseline
    if kk>0:
        if use_basil_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_inhibition_noise_apic_up)]
                data_soma = [np.hstack(data_inhibition_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_inhibition
                peaks_bspk_loc = peaks_bspk_loc_inhibition
                nspk_peak = nspk_peak_inhibition
            elif kk == 2: 
                data_apic_up = [np.hstack(data_cancellation_noise_apic_up)]
                data_soma = [np.hstack(data_cancellation_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_cancellation
                peaks_bspk_loc = peaks_bspk_loc_cancellation
                nspk_peak = nspk_peak_cancellation
        elif use_apic_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_apic_inh_noise_apic_up)]
                data_soma = [np.hstack(data_apic_inh_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_apic_inh
                peaks_bspk_loc = peaks_bspk_loc_apic_inh
                nspk_peak = nspk_peak_apic_inh
            elif kk == 2: 
                data_apic_up = [np.hstack(data_apic_can_noise_apic_up)]
                data_soma = [np.hstack(data_apic_can_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_apic_can
                peaks_bspk_loc = peaks_bspk_loc_apic_can
                nspk_peak = nspk_peak_apic_can
        elif use_soma_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_soma_inh_noise_apic_up)]
                data_soma = [np.hstack(data_soma_inh_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_soma_inh
                peaks_bspk_loc = peaks_bspk_loc_soma_inh
                nspk_peak = nspk_peak_soma_inh
            elif kk == 2: 
                data_apic_up = [np.hstack(data_soma_can_noise_apic_up)]
                data_soma = [np.hstack(data_soma_can_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_soma_can
                peaks_bspk_loc = peaks_bspk_loc_soma_can
                nspk_peak = nspk_peak_soma_can

        
        
    if show_apic:
        if kk == 0 or kk == 1:
            min_pk_nsp = -60
        elif kk == 2:
            min_pk_nsp = -58.5
            
        spk_times_all = turn_trace_to_spk(data_apic_up, min_hight_nspk = min_pk_nsp, min_hight_bspk = -47.5,\
                                          prom_nspk = 1.5, prom_bspk = 15)
        peaks_nspk_loc_apic_up = np.concatenate(spk_times_all['peaks_nspk_all'])
        peaks_bspk_loc = (spk_times_all['peaks_bspk_all'])
        nspk_peaks_all_apic_up = np.concatenate([x['peak_heights'] for x in spk_times_all['properties_nspk_all']])
        bspk_peaks_all = ([x['peak_heights'] for x in spk_times_all['properties_bspk_all']])
        
        data_rel = np.concatenate(data_apic_up)
        PKS = nspk_peaks_all_apic_up
        peaks_loc_rel = peaks_nspk_loc_apic_up    
    
    else:
        data_rel = np.concatenate(data_soma)

        
        PKS = np.concatenate(nspk_peak) 
        peaks_loc_rel = np.concatenate(peaks_nspk_loc)
        
       

    fst_dev = np.diff(data_rel,1)
    fst_dev = np.insert(fst_dev,0, 0)

    mp_nspk_locs = []
    idx_list = []
    for ii in range(len(peaks_loc_rel)):  
        rel_idx = [peaks_loc_rel[ii]-time_before_pk+add_ind+np.where(fst_dev[peaks_loc_rel[ii]-time_before_pk:peaks_loc_rel[ii]]>min_der)[0]]
        if len(rel_idx[0])>0:
            mp_nspk_locs.append(rel_idx[0][0])
        else:
            print(ii)
            idx_list.append(ii)

            PKS = np.delete(PKS,idx_list)  
    peaks_loc_rel = np.delete(peaks_loc_rel,idx_list)  

    MP = data_rel[mp_nspk_locs]
    AMP = PKS-MP
    
    mean_AMP_MP_all[kk,0] = np.mean(PKS)
    mean_AMP_MP_all[kk,1] = np.mean(MP)

    ns_pk_non_evoking = []
    ns_mp_non_evoking = []
    ns_amp_non_evoking = []
    ns_loc_non_evoking = []
    ns_pk_pre = []
    ns_mp_pre = []
    ns_amp_pre = []
    ns_loc_pre_bs = []
    if any(peaks_bspk_loc[0]):
        nspk_before_idx_all = []
        for jj in range(len(peaks_bspk_loc[0])):
            nspk_before_idx = np.concatenate(np.where((peaks_loc_rel > (peaks_bspk_loc[0][jj] - nspk_before_bspk)) & (peaks_loc_rel < peaks_bspk_loc[0][jj])))
            if len(nspk_before_idx) > 0:
                nspk_before_idx = nspk_before_idx[-1]
                nspk_before_idx_all.append(nspk_before_idx)
        idx_all = range(len(PKS))
        nspk_non_before_idx = np.delete(idx_all,nspk_before_idx_all) 
        ns_pk_pre.append(PKS[nspk_before_idx_all])
        ns_loc_pre_bs.append(peaks_loc_rel[nspk_before_idx_all])
        ns_pk_non_evoking.append(PKS[nspk_non_before_idx])    
        ns_loc_non_evoking.append(peaks_loc_rel[nspk_non_before_idx])  

        ns_mp_pre.append(MP[nspk_before_idx_all])
        ns_mp_non_evoking.append(MP[nspk_non_before_idx])    
        ns_amp_pre.append(AMP[nspk_before_idx_all])
        ns_amp_non_evoking.append(AMP[nspk_non_before_idx])        

        ns_mp_pre_conc = np.concatenate(ns_mp_pre);
        ns_amp_pre_conc = np.concatenate(ns_amp_pre);
        ns_pk_pre_conc = np.concatenate(ns_pk_pre);
        ns_mp_non_evoking_conc = np.concatenate(ns_mp_non_evoking)
        ns_amp_non_evoking_conc = np.concatenate(ns_amp_non_evoking)
        ns_pk_non_evoking_conc = np.concatenate(ns_pk_non_evoking)

        nbins = np.ceil((np.max(MP)-np.min(MP))/bin_size)
        edges = np.linspace(np.min(MP),np.max(MP),num = int(nbins+1))
        inds_non_evoke = np.digitize(np.array(ns_mp_non_evoking_conc ,dtype='float'), bins=edges)
        mean_pk_non_evoke = np.empty((np.size(edges)))
        mean_pk_non_evoke[:] = np.NaN
        for ii in range(len(edges)):
            if np.sum(inds_non_evoke == ii) > min_ind:
                mean_pk_non_evoke[ii] =  np.mean(ns_pk_non_evoking_conc[inds_non_evoke == ii])                  
        inds_evoke = np.digitize(np.array(ns_mp_pre_conc ,dtype='float'), bins=edges)
        mean_pk_evoke = np.empty((np.size(edges)))
        mean_pk_evoke[:] = np.NaN
        for ii in range(len(edges)):
            if np.sum(inds_evoke == ii) > min_ind:
                mean_pk_evoke[ii] =  np.mean(ns_pk_pre_conc[inds_evoke == ii])                  
        
    else:
        nbins = np.ceil((np.max(MP)-np.min(MP))/bin_size)
        edges = np.linspace(np.min(MP),np.max(MP),num = int(nbins+1))
        counts_all, bins_all = np.histogram(np.array(MP,dtype='float'),bins=edges)
        inds_non_evoke = np.digitize(np.array(MP,dtype='float'), bins=edges)
        mean_pk_non_evoke = np.empty((np.size(edges)))
        mean_pk_non_evoke[:] = np.NaN
        for ii in range(len(edges)):
            if np.sum(inds_non_evoke == ii) > min_ind:
                mean_pk_non_evoke[ii] =  np.mean(PKS[inds_non_evoke == ii])                  
        mean_pk_evoke = np.empty((np.size(edges)))
        mean_pk_evoke[:] = np.NaN
        ind_evoke = np.empty((np.size(edges)))
    
    
    
    X_MP  = np.transpose([MP])
    model_cell = LinearRegression(fit_intercept=True).fit(X_MP, np.transpose(PKS))
    
    plt.scatter(edges[4:54],mean_pk_non_evoke[4:54],s=10, color='k')
    plt.scatter(edges[4:54],mean_pk_evoke[4:54],s=10, facecolors='none', edgecolors='k')
    plt.legend(['non-evoking','evoking'],frameon = False, prop={'size': fs})
    plt.tight_layout()
    plt.ylabel('peak (mV)', fontsize = fs)
    plt.xlabel('baseline membrane potential (mV)', fontsize = fs)

    ax = plt.gca()
    ax.tick_params(direction='in', length=2, width=0.5, colors='k',
                   grid_color='k', grid_alpha=1)

plt.show()


# H-H analysis

In [28]:
percentage_evoking_ramp = np.zeros(3)

mean_peak = np.zeros((3,3))
std_peak_all = np.zeros((3,3))

mean_peak_evoking = np.zeros((3,3))
std_peak_evoking = np.zeros((3,3))

mean_peak_non_ramp = np.zeros((3,3))
std_peak_non_ramp = np.zeros((3,3))

titles = ['baseline','cancellation','inhibition']
thresh_val_all = np.zeros(3)
thresh_val_all_apic_up = np.zeros(3)

p_thresh_all = np.zeros(3)
p_thresh_all_apic_up = np.zeros(3)
ratio_all = np.zeros(3)
nr_nspk = np.zeros(3)
nr_bspk = np.zeros(3)

dt = 0.025
loc_up_apic_up = 0.65/dt

for kk in range(1):

    if kk == 0:
        data_apic = [np.hstack(data_baseline_noise)]
        data_apic_up = [np.hstack(data_baseline_noise_apic_up)]
        data_soma = [np.hstack(data_baseline_noise_soma)]
        data_ina = [np.hstack(data_baseline_ina)] 
        data_ik = [np.hstack(data_baseline_ik)] 
        data_m = [np.hstack(data_baseline_m)] 
        data_h = [np.hstack(data_baseline_h)] 
        data_n = [np.hstack(data_baseline_n)]

    elif kk == 1:    
        data_apic = [np.hstack(data_cancellation_noise)]
        data_apic_up = [np.hstack(data_cancellation_noise_apic_up)]
        data_soma = [np.hstack(data_cancellation_noise_soma)]
        data_ina = [np.hstack(data_cancellation_ina)] 
        data_ik = [np.hstack(data_cancellation_ik)] 
        data_m = [np.hstack(data_cancellation_m)] 
        data_h = [np.hstack(data_cancellation_h)] 
        data_n = [np.hstack(data_cancellation_n)]
    elif kk == 2:    
        data_apic = [np.hstack(data_inhibition_noise)]
        data_apic_up = [np.hstack(data_inhibition_noise_apic_up)]
        data_soma = [np.hstack(data_inhibition_noise_soma)]
        data_ina = [np.hstack(data_inhibition_ina)] 
        data_ik = [np.hstack(data_inhibition_ik)] 
        data_m = [np.hstack(data_inhibition_m)] 
        data_h = [np.hstack(data_inhibition_h)] 
        data_n = [np.hstack(data_inhibition_n)]
        
    spk_times_all = turn_trace_to_spk(data_soma, min_hight_nspk = -59, min_hight_bspk = -47.5,\
                                      prom_nspk = 3, prom_bspk = 15)
    
    nr_bspk_two_nspk = 0
    
    ns_pk_non_evoking = []
    ns_loc_non_evoking = []
    nspk_peaks_pre = []
    ns_loc_pre_bs = []
    dt = 0.025
    nspk_before_bspk = np.round(8/dt)
    peaks_nspk_loc = spk_times_all['peaks_nspk_all']
    peaks_bspk_loc = spk_times_all['peaks_bspk_all']
    nspk_peak = [x['peak_heights'] for x in spk_times_all['properties_nspk_all']]
    nspk_peaks_all = np.concatenate(nspk_peak)
    
    peaks_nspk_loc = np.concatenate(peaks_nspk_loc)
    
    fst_dev = np.diff(np.concatenate(data_apic_up),1)
    fst_dev = np.insert(fst_dev,0, 0)

    mp_nspk_locs = []
    idx_list = []
    for ii in range(len(peaks_nspk_loc)):  
        rel_idx = [peaks_nspk_loc[ii]-time_before_pk+add_ind+np.where(fst_dev[peaks_nspk_loc[ii]-time_before_pk:peaks_nspk_loc[ii]]>min_der)[0]]
        if len(rel_idx[0])>0:
            mp_nspk_locs.append(rel_idx[0][0])
        else:
            print(ii)
            idx_list.append(ii)
#         
    nspk_peaks_all = np.delete(nspk_peaks_all,idx_list)  
    peaks_nspk_loc = np.delete(peaks_nspk_loc,idx_list)  
        
#     mp_nspk_loc = np.array([peaks_nspk_loc[ii]-time_before_pk-1+np.where(fst_dev[peaks_nspk_loc[ii]-time_before_pk:peaks_nspk_loc[ii]]>min_der)[0][0] for ii in range(len(peaks_nspk_loc))],dtype=int)
    MP = data_apic_up[0][mp_nspk_locs]
    AMP = nspk_peaks_all-MP
 
    
    if any(peaks_bspk_loc[0]):
        nspk_before_idx_all = []
        for jj in range(len(peaks_bspk_loc[0])):
            nspk_before_idx = np.concatenate(np.where((peaks_nspk_loc > (peaks_bspk_loc[0][jj] - nspk_before_bspk)) & (peaks_nspk_loc < peaks_bspk_loc[0][jj])))
            if len(nspk_before_idx) > 0:
                if len(nspk_before_idx) > 1:
                    nr_bspk_two_nspk = nr_bspk_two_nspk+1    
                nspk_before_idx = nspk_before_idx[-1]
                nspk_before_idx_all.append(nspk_before_idx)
            else:
                print('bad')

    idx_all = range(len(nspk_peaks_all))
    nspk_non_before_idx = np.delete(idx_all,nspk_before_idx_all) 
    nspk_peaks_pre.append(nspk_peak[0][nspk_before_idx_all])
    ns_loc_pre_bs.append(peaks_nspk_loc[nspk_before_idx_all])
    ns_pk_non_evoking.append(nspk_peak[0][nspk_non_before_idx])    
    ns_loc_non_evoking.append(peaks_nspk_loc[nspk_non_before_idx])  
            
                     

    nr_nspk[kk] = len(peaks_nspk_loc)
    print('nr nspk = ', nr_nspk[kk])
    nr_bspk[kk] = len(np.concatenate(peaks_bspk_loc))
    print('nr bspk = ', nr_bspk[kk])
    
    thresh = 0.1
    
    thresh_val_all[kk] = np.quantile(nspk_peaks_pre,thresh)

    above_thresh_idx = np.where(nspk_peaks_all > thresh_val_all[kk])
    evoke_idx = nspk_before_idx_all
#     evoke_idx = np.intersect1d(above_thresh_idx,nspk_before_idx_all,assume_unique=True)
    non_evoke_idx = np.setdiff1d(above_thresh_idx,nspk_before_idx_all, assume_unique=True)
    
#     evoke_idx = np.asarray(nspk_before_idx_all)
#     non_evoke_idx = nspk_non_before_idx

    peak_evoke = nspk_peaks_all[evoke_idx]
    peak_non_evoke = nspk_peaks_all[non_evoke_idx]
    peak_above_thresh = nspk_peaks_all[above_thresh_idx]

    amp_evoke = AMP[evoke_idx]
    amp_non_evoke = AMP[non_evoke_idx]
    amp_above_thresh = AMP[above_thresh_idx]

    ina_peaks_all = data_ina[0][peaks_nspk_loc+int(loc_up_apic_up)]
    ik_peaks_all = data_ik[0][peaks_nspk_loc+int(loc_up_apic_up)]
    m_peaks_all = data_m[0][peaks_nspk_loc+int(loc_up_apic_up)]
    h_peaks_all = data_h[0][peaks_nspk_loc+int(loc_up_apic_up)]
    n_peaks_all = data_n[0][peaks_nspk_loc+int(loc_up_apic_up)]

    ina_evoke = ina_peaks_all[evoke_idx]
    ina_non_evoke = ina_peaks_all[non_evoke_idx]    
    ik_evoke = ik_peaks_all[evoke_idx]
    ik_non_evoke = ik_peaks_all[non_evoke_idx]
    m_evoke = m_peaks_all[evoke_idx]
    m_non_evoke = m_peaks_all[non_evoke_idx]
    m_above_thresh = m_peaks_all[above_thresh_idx]
    h_evoke = h_peaks_all[evoke_idx]
    h_non_evoke = h_peaks_all[non_evoke_idx]
    h_above_thresh = h_peaks_all[above_thresh_idx]
    n_evoke = n_peaks_all[evoke_idx]
    n_non_evoke = n_peaks_all[non_evoke_idx]
    n_above_thresh = n_peaks_all[above_thresh_idx]

    ### fits
    
    evoke = np.column_stack((m_evoke,h_evoke))
    a = np.ones(len(m_evoke),dtype=int)
    non_evoke = np.column_stack((m_non_evoke,h_non_evoke))
    b = np.zeros(len(m_non_evoke),dtype=int)
    all_mh = np.concatenate((evoke,non_evoke))
    all_clas = np.concatenate((a,b))
    
    X = all_mh
    y = all_clas




nr nspk =  2555.0
nr bspk =  108.0


In [42]:
slope_pk = np.zeros(2)
slope_amp = np.zeros(2)

%matplotlib osx

fig = plt.figure(figsize=(3,3))

fs = 6
ts = 6
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs)

x_range = [-62,-45]
y_range = [0,0.15]
if kk == 0:
    x_min, x_max = X[:, 0].min() - .01, X[:, 0].max() + .01
    y_min, y_max = X[:, 1].min() - .01, X[:, 1].max() + .03
    h = .02 # step size
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

if kk == 0:    
    clf = SVC(gamma='scale',C=1000)
    clf.fit(X, y)
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

cm1 = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

score = clf.score(X, y)
ax = fig.add_subplot(1,1,1)
ms = 3
ax.contourf(xx, yy, Z, cmap=cm1, alpha=.8)  
perc_use = 0.3
non_evoke_idx = np.random.randint(0,len(m_non_evoke),int(perc_use*len(m_non_evoke)))
evoke_idx = np.random.randint(0,len(m_evoke),int(perc_use*len(m_evoke)))

plt.scatter(m_non_evoke[non_evoke_idx],h_non_evoke[non_evoke_idx], facecolor = 'gray',marker = 'x',s = ms)
plt.scatter(m_evoke[evoke_idx],h_evoke[evoke_idx],marker = 'o',s = ms-1,facecolors='none', edgecolors='r')
ax.text(xx.max() - .01, yy.min() + .02, ('%.2f' % score).lstrip('0'),
        size=fs, horizontalalignment='right')


ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())

plt.title(titles[kk], fontsize = fs)
ax.set_xlabel('m', fontsize = fs)
ax.set_ylabel('h', fontsize = fs)
ax = plt.gca()
ax.tick_params(direction='in', length=2, width=0.5, colors='k',
               grid_color='k', grid_alpha=1)

data1 = peak_above_thresh
data2 = m_above_thresh
X_M  = np.transpose([data1])
model_cell = LinearRegression(fit_intercept=True).fit(X_M, np.transpose(data2))
slope_pk[0] = model_cell.coef_[0]

data1 = amp_above_thresh
data2 = m_above_thresh
X_M  = np.transpose([data1])
model_cell = LinearRegression(fit_intercept=True).fit(X_M, np.transpose(data2))
slope_amp[0] = model_cell.coef_[0]

data1 = peak_above_thresh
data2 = h_above_thresh
X_M  = np.transpose([data1])
model_cell = LinearRegression(fit_intercept=True).fit(X_M, np.transpose(data2))
slope_pk[1] = model_cell.coef_[0]

data1 = amp_above_thresh
data2 = h_above_thresh
X_M  = np.transpose([data1])
model_cell = LinearRegression(fit_intercept=True).fit(X_M, np.transpose(data2))
slope_amp[1] = model_cell.coef_[0]


In [43]:
colr = ['c','m']
titles = ['m','h']

fig = plt.figure(figsize=(6,4))


plt.subplot(1,2,1)
plt.bar(titles,slope_pk,color=colr,width = 0.5)
plt.ylabel('slope, peak', fontsize = fs)
ax = plt.gca()
ax.fontsize = fs
ax.tick_params(direction='in', length=2, width=0.5, colors='k',
               grid_color='k', grid_alpha=1)

 
plt.subplot(1,2,2)
plt.bar(titles,slope_amp,color=colr,align='center',width = 0.5)
plt.ylabel('slope, amplitude', fontsize = fs)
ax = plt.gca()
ax.fontsize = fs
ax.tick_params(direction='in', length=2, width=0.5, colors='k',
               grid_color='k', grid_alpha=1)


# attenuation as a function of amplitude

In [46]:
# %matplotlib osx

nr_hist = 200
x_range = [-65,-52]
y_range = [0,0.15]
min_ind = 1

titles = ['baseline','inhib.','cancel.']
sites = ['soma','apic','apic up']
colr = ['k','r','m']

dt = 0.025
peak_fit_all = {}
lnspc_peak_fit_all = {}
height_fit_all = {}
lnspc_height_fit_all = {}

use_basil_inh = True
use_soma_inh = False
use_apic_inh = False

slp_bias_all = np.zeros((3,2))

for kk in range(1):
    
    if kk == 0:
        data_apic_up = [np.hstack(data_baseline_noise_apic_up)]
        data_soma = [np.hstack(data_baseline_noise_soma)]
        peaks_nspk_loc = peaks_nspk_loc_baseline
        peaks_bspk_loc = peaks_bspk_loc_baseline
        nspk_peak = nspk_peak_baseline
    if kk>0:
        if use_basil_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_inhibition_noise_apic_up)]
                data_soma = [np.hstack(data_inhibition_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_inhibition
                peaks_bspk_loc = peaks_bspk_loc_inhibition
                nspk_peak = nspk_peak_inhibition
            elif kk == 2: 
                data_apic_up = [np.hstack(data_cancellation_noise_apic_up)]
                data_soma = [np.hstack(data_cancellation_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_cancellation
                peaks_bspk_loc = peaks_bspk_loc_cancellation
                nspk_peak = nspk_peak_cancellation
        elif use_apic_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_apic_inh_noise_apic_up)]
                data_soma = [np.hstack(data_apic_inh_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_apic_inh
                peaks_bspk_loc = peaks_bspk_loc_apic_inh
                nspk_peak = nspk_peak_apic_inh
            elif kk == 2: 
                data_apic_up = [np.hstack(data_apic_can_noise_apic_up)]
                data_soma = [np.hstack(data_apic_can_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_apic_can
                peaks_bspk_loc = peaks_bspk_loc_apic_can
                nspk_peak = nspk_peak_apic_can
        elif use_soma_inh:    
            if kk == 1: 
                data_apic_up = [np.hstack(data_soma_inh_noise_apic_up)]
                data_soma = [np.hstack(data_soma_inh_noise_soma)]  
                peaks_nspk_loc = peaks_nspk_loc_soma_inh
                peaks_bspk_loc = peaks_bspk_loc_soma_inh
                nspk_peak = nspk_peak_soma_inh
            elif kk == 2: 
                data_apic_up = [np.hstack(data_soma_can_noise_apic_up)]
                data_soma = [np.hstack(data_soma_can_noise_soma)]
                peaks_nspk_loc = peaks_nspk_loc_soma_can
                peaks_bspk_loc = peaks_bspk_loc_soma_can
                nspk_peak = nspk_peak_soma_can


    if kk == 0 or kk == 1:
        min_pk_nsp = -60
    elif kk == 2:
        min_pk_nsp = -58.5

    spk_times_all = turn_trace_to_spk(data_apic_up, min_hight_nspk = min_pk_nsp, min_hight_bspk = -47.5,\
                                      prom_nspk = 1.5, prom_bspk = 15)
    peaks_nspk_loc_apic_up = (spk_times_all['peaks_nspk_all'])
    nspk_peaks_all_apic_up = np.concatenate([x['peak_heights'] for x in spk_times_all['properties_nspk_all']])

    pks_apic = (nspk_peaks_all_apic_up) 
    peaks_loc_apic = np.concatenate(peaks_nspk_loc_apic_up)
    peaks_nspk_loc_apic_up = np.concatenate(peaks_nspk_loc_apic_up)
    
    fst_dev_apic = np.diff(np.concatenate(data_apic_up),1)
    fst_dev_apic = np.insert(fst_dev_apic,0, 0)


    mp_nspk_loc_apic = []
    idx_list = []
    for ii in range(len(peaks_loc_apic)):  
        rel_idx = [peaks_loc_apic[ii]-time_before_pk+add_ind+np.where(fst_dev[peaks_loc_apic[ii]-time_before_pk:peaks_loc_apic[ii]]>min_der)[0]]
        if len(rel_idx[0])>0:
            mp_nspk_loc_apic.append(rel_idx[0][0])
        else:
            print(ii)
            idx_list.append(ii)

    pks_apic = np.delete(pks_apic,idx_list)  
    peaks_loc_apic = np.delete(peaks_loc_apic,idx_list)  

    MP_apic = np.concatenate(data_apic_up)[mp_nspk_loc_apic]
    AMP_apic = pks_apic-MP_apic    
        
    pks_soma = np.concatenate(nspk_peak) 
    peaks_loc_soma = np.concatenate(peaks_nspk_loc)
        
    fst_dev_soma = np.diff(np.concatenate(data_soma),1)
    fst_dev_soma = np.insert(fst_dev_soma,0, 0)

    mp_nspk_loc_soma = np.array([peaks_loc_soma[ii]-time_before_pk+add_ind+np.where(fst_dev_soma[peaks_loc_soma[ii]-time_before_pk:peaks_loc_soma[ii]]>min_der)[0][0] for ii in range(len(peaks_loc_soma))],dtype=int)
    MP_soma = np.concatenate(data_soma)[mp_nspk_loc_soma]
    AMP_soma = pks_soma-MP_soma    
    

    idx_all = range(len(MP_soma))
    idx_apic_soma = []
    idx_apic = []
    for jj in range(len(MP_soma)):
        apic_f_som = np.concatenate(np.where((peaks_loc_apic > peaks_loc_soma[jj]) & (peaks_loc_apic < peaks_loc_soma[jj]+15)))
        if len(apic_f_som)>0:
#         if len(np.where(([(a > peaks_loc_soma[jj] and a < peaks_loc_soma[jj]+15) for a in peaks_loc_apic]))[0])>0:
            idx_apic_soma.append(jj) 
            idx_apic.append(apic_f_som[0]) 
    if len(idx_apic_soma) != len(peaks_loc_apic):
        print('mismatch')
    
 
    bin_size = 0.1
    AMP_som_rel = AMP_soma[idx_apic_soma]
    AMP_apic_rel = AMP_apic[idx_apic]
    
    nbins = np.ceil((np.max(AMP_som_rel)-np.min(AMP_som_rel))/bin_size)
    edges = np.linspace(np.min(AMP_som_rel),np.max(AMP_som_rel),num = int(nbins+1))
    counts_all, bins_all = np.histogram(np.array(AMP_som_rel,dtype='float'),bins=edges)
    inds_amp_som = np.digitize(np.array(AMP_som_rel,dtype='float'), bins=edges)
    mean_amp_apic = np.empty((np.size(edges)))
    mean_amp_apic[:] = np.NaN
    for ii in range(len(edges)):
        if np.sum(inds_amp_som == ii) > min_ind:
            mean_amp_apic[ii] =  np.mean(AMP_apic_rel[inds_amp_som == ii])                  

    X_MP  = (AMP_som_rel.reshape(-1, 1))
    model_cell = LinearRegression(fit_intercept=True).fit(X_MP, (AMP_som_rel-AMP_apic_rel).reshape(-1, 1))


In [48]:
fig = plt.figure(figsize=(7,3))

fs = 6
ts = 6
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs)

plt.subplot(1,2,1)
plt.scatter(edges,mean_amp_apic,s=10, color = colr[kk])
plt.ylabel('AMP apic (mV)', fontsize = fs)
plt.xlabel('amplitude, recorded in soma (mV)', fontsize = fs)

plt.subplot(1,2,2)
plt.scatter(edges,edges-mean_amp_apic,s=10, color = colr[kk])
plt.ylabel('attenuation amount (mV)', fontsize = fs)
plt.xlabel('amplitude, recorded in soma (mV)', fontsize = fs)

plt.show()
